#### 🔰Cet projet consiste à prédire si un client de Telecom va se désabonner ou pas. Ceci est une notebook de recherche qui nous servira à faire l'analyse exploratoire des données de l'entreprise afin de pouvoir créer un modèle de machine learning capable de prédire si un client se désabonnera ou non.

#### ⚜💢Je me nomme Chrystal Orian VIGAN. Je suis datascientiste. Email: viganchrystal@gmail.com

##### ⏸Début de projet: 18/03/2024

### 📑Steps
 #####  1- Analyse exploratoire des données (EDA)
 #####  2- Pre-traitement des donnees (Preprocessing)
 #####  3- Création des modèles de ML

In [ ]:
# Importation des modules

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from sklearn.model_selection import train_test_split, GridSearchCV, learning_curve
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline
from sklearn.compose import make_column_transformer 
from sklearn.metrics import RocCurveDisplay, classification_report, confusion_matrix, f1_score
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier

import ipywidgets as widgets
from ipywidgets import interact, interactive

import joblib


In [ ]:
# IMPORTATION DE NOTRE DATASET

Data = pd.read_csv('./data/WA_Fn-UseC_-Telco-Customer-Churn.csv')

### EDA

In [ ]:
df = Data.copy()

df.head()

In [ ]:
print('Shape', df.shape)
print('\n')
print(df.describe())

In [ ]:
print(df.dtypes.value_counts())
print('\n')
print(df.dtypes.value_counts().plot.pie())

In [ ]:
df['customerID'].nunique() == df.shape[0]

# #Nous constatons que la colonne 'customerID' est l'identifiant de chaque client au 
# niveau de l'entreprise. Alors elle ne nous apportera aucune information dans l'analyse 
# et la prédiction dans ce cas

In [ ]:
df = df.drop('customerID', axis=1)

print(df.shape)
print('\n')
df.columns.to_list()

In [ ]:
df.isna().sum()

# Nous constatons qu'il y a pas de valeur manquante dans notre dataframe

In [ ]:
seed = 123

trainset, test = train_test_split(df, test_size=0.3, random_state=seed, stratify=df['Churn'])
test, validate = train_test_split(test, test_size=0.5, random_state=seed, stratify=test['Churn'])

In [ ]:
# Enregistrement des Jeux de données

trainset.to_csv('./data/train.csv', index=False)
test.to_csv('./data/test.csv', index=False)
validate.to_csv('./data/validate.csv', index=False)

In [ ]:
train = trainset.copy()

In [ ]:
categorical = train.select_dtypes(include='object')
numerical = train.select_dtypes(exclude='object')

In [ ]:
for col in categorical:
    print(f'{col :-<50} {train[col].unique()}')

 Nous constatons que la variable 'TotalCharges' est de types float mais la machine la considère comme une varibles de types objet....
 Nous allons bien l'analyser et voir la raison

In [ ]:
pd.to_numeric(df['TotalCharges'], errors='raise')
#Ce code avec son argument errors="raise" nous a permis de savoirs par exemple qu'à la ligne 488, il y a aucune valeurs "  " . Ce qui traduit pourquoi la machine a considérer
# la variable 'TotalCharge' comme étant une variables de types objets car il y a beaucoup de données vide. Avec le même code et sont arguments errors="coerce", nous allons 
# convertir cette valeur manquante en NaN. Nous feront ceci dans l'étape de preprocessing.

df['TotalCharges'][484:493]

In [ ]:
# Diagramme a barre de chaque variables quantitatives

for col in numerical:
    plt.figure()
    sns.displot(train[col])
    plt.legend(col)
    plt.show()
    plt.close


In [ ]:
for col in categorical:
    plt.figure()
    sns.countplot(data=train, x=col)
    plt.legend(col)
    plt.show()
    plt.close()

In [ ]:
for col in categorical:
    plt.figure()
    sns.heatmap(pd.crosstab(train['Churn'],
                            train[col]), annot=True, fmt='d')
    plt.legend()
    plt.show()
    plt.close()

In [ ]:
sns.heatmap(numerical.corr(), annot=True, cmap="RdBu", fmt='2g')

In [ ]:
sns.clustermap(numerical.corr(), annot=True)

In [ ]:
for col in categorical:
    print(f'{col:-<50} {train[col].unique()}')

In [ ]:
for col in categorical:
    plt.figure()
    train[col].value_counts().plot.pie()
    plt.title(col)

## Preprocessing

In [ ]:
train['Churn'].value_counts(normalize=True)

# Nous constatons qu'il y a une forte déséquilibre de classe dans notre variable cible

In [ ]:
# Nous allons traiter la variable TotalCharges

def convert_total_charge(df, column_name):
    '''
    Cette nous permettra de convertir les lignes vides de la variable TotalCharges en NaN et donc la vonvertir en float

    input:
          df = dataset
          column_name = 'TotalCharges'
    output:
          df modifie
    '''
    df[column_name] = df[column_name].replace(' ', np.nan).astype(float)
    return df

In [ ]:
train = convert_total_charge(train, 'TotalCharges')
valide = convert_total_charge(validate, 'TotalCharges')

In [ ]:
def cut_x_y(df):
    y = df['Churn']
    x = df.drop('Churn', axis=1)

    return x, y

In [ ]:
train.dtypes.value_counts()

In [ ]:
#churn = df['Churn']
numerical_df = train.select_dtypes(exclude='object')
categorical_df = train.select_dtypes(include='object')

In [ ]:
# Pileline de preprocessing

numerical_transformer = make_pipeline(('imputer', SimpleImputer(strategy='most_frequent')),
                                      ('standard', StandardScaler()))

categorical_transformer = make_pipeline('encoder', OneHotEncoder())

In [ ]:
preprocessing = make_column_transformer((numerical_transformer, numerical_df),
                                        (categorical_transformer, categorical_df))

## Modelisation

In [ ]:
RandomForest = make_pipeline(preprocessing, RandomForestClassifier(random_state=seed))

KNN = make_pipeline(preprocessing, KNeighborsClassifier())

SVM = make_pipeline(preprocessing, SVC(random_state=seed))

ADA = make_pipeline(preprocessing, AdaBoostClassifier(random_state=seed))

Log_regre = make_pipeline(preprocessing, LogisticRegression(random_state=seed))

In [ ]:
dict_model = {'RandomForestClassifier' : RandomForest,
              'KNeighborsClassifier': KNN,
              'SVC': SVM,
              'AdaBoostClassifier' : ADA,
              'LogisticRegression' : Log_regre}

In [ ]:
x_train, y_train = cut_x_y(train)
x_val, y_val = cut_x_y(valide)

In [ ]:
def evaluation(model):
    model.fit(x_train, y_train)
    y_val_pred = model.predict(x_val)

    print(confusion_matrix(y_val, y_val_pred))
    print(classification_report(y_val, y_val_pred))

    N, train_score, val_score = learning_curve(model , x_train, y_train, cv=4,
                                               scoring='f1_score', train_sizes=np.linspace(0.1, 1, 10))
    
    rf_roc = RocCurveDisplay.from_estimator(model, x_val, y_val)
    
    plt.figure(figsize=(12, 8))
    print(rf_roc)
    plt.plot(N, train_score.mean(axis=1), label='train score')
    plt.plot(N, val_score.mean(axis=1), label='validation score')
    plt.legend()

In [ ]:
for name, model in dict_model.items():
    print(name)
    evaluation(model)

In [ ]:
print('hollo x')